In [2]:
%load_ext sql
import os
from sqlalchemy import create_engine

pgconfig = {
    'host': 'db',
    'port': os.environ['PG_PORT'],
    'database': os.environ['PG_DATABASE'],
    'user': os.environ['PG_USER'],
    'password': os.environ['PG_PASSWORD'],
}
dsl = 'postgresql://{user}:{password}@{host}:{port}/{database}'.format(**pgconfig)
conn = create_engine(dsl)

# MagicコマンドでSQLを書くための設定
%sql conn

In [10]:
%%sql
drop table if exists tbl;
create table tbl (
    id integer primary key,
    group_id integer not null,
    value integer not null
);

insert into tbl
values (1,1,2),
    (2,1,4),
    (3,1,2),
    (4,1,6),
    (5,1,4),
    (6,2,4),
    (7,2,3),
    (8,3,10);

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
Done.
8 rows affected.


[]

In [16]:
%%sql
select max(t1.id), t1.group_id, max(t1.value)
from tbl as t1, tbl as t2
where t1.group_id = t2.group_id and
t1.value <= t2.value
group by t1.group_id, t1.value
having count(*) <= 2

*  postgresql://padawan:***@db:5432/dsdojo_db
4 rows affected.


max,group_id,max_1
7,2,3
4,1,6
6,2,4
8,3,10


In [ ]:
%%sql
select *
from (
    select *,
        dense_rank() over (
            partition by group_id
            order by value desc
        ) as rnk
    from tbl
) as tmp
where rnk <= 2